In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import math

cap = cv2.VideoCapture(0)
hand_detector = mp.solutions.hands.Hands(max_num_hands=1)
drawing_utils = mp.solutions.drawing_utils
screen_width, screen_height = pyautogui.size()

index_x = index_y = 0
thumb_x = thumb_y = 0

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    height, width, _ = frame.shape

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    output = hand_detector.process(rgb_frame)
    hands = output.multi_hand_landmarks

    if hands:
        for hand in hands:
            drawing_utils.draw_landmarks(frame, hand)
            landmarks = hand.landmark

            for id, landmark in enumerate(landmarks):
                x = int(landmark.x * width)
                y = int(landmark.y * height)

                if id == 8:  # Index finger
                    index_x = screen_width / width * x
                    index_y = screen_height / height * y
                    cv2.circle(frame, (x, y), 10, (255, 0, 255), -1)
                    pyautogui.moveTo(index_x, index_y)

                if id == 4:  # Thumb
                    thumb_x = screen_width / width * x
                    thumb_y = screen_height / height * y
                    cv2.circle(frame, (x, y), 10, (0, 255, 0), -1)

            # Check distance between index and thumb
            dist = math.hypot(thumb_x - index_x, thumb_y - index_y)
            if dist < 40:
                pyautogui.click()
                pyautogui.sleep(1)  # prevent multiple clicks

    cv2.imshow("Virtual Mouse", frame)

    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()
